In [1]:
from typing import List
import pandas as pd
import scanpy as sc
from anndata import AnnData


def sql_query(query, fetch_results=True):
    # postgres data retrieval with consistent output, both in the jupyter development
    # environment (plpy is not available) and at runtime inside a plpython3u stored procedure
    try:
        import plpy
    except:
        from postgres_utils import engine
        from sqlalchemy import text

        with engine.connect() as connection:
            r = connection.execute(text(query))
            if fetch_results:
                return [row._mapping for row in r.fetchall()]
            else:
                return
    r = plpy.execute(query)
    if fetch_results:
        return [row for row in r]
    else:
        return


def read_h5ad(study_id: int):
    filename = sql_query(f"select filename from study where study_id = {study_id}")[0]["filename"]
    try:
        import plpy

        # running in postgres docker image
        filename = f"/h5ad_store/{filename}"
    except:
        # running in devenv jupyter notebook
        filename = f"../scratch/{filename}"
    return sc.read(filename)


# def add_custom_annotation(adata: AnnData, study_id:int, annotation_group_id:int):


def get_annotation_df(study_id: int, annotation_group_id: int):
    annotation_data = sql_query(
        f"""
       SELECT ssa.annotation_value_id, ss.h5ad_obs_index
     FROM study_sample_annotation ssa
         join annotation_value av on av.annotation_value_id = ssa.annotation_value_id
     cross join UNNEST(study_sample_ids) as sample_id
     join study_sample ss on ss.study_id = ssa.study_id and ss.study_sample_id = sample_id
            WHERE ssa.study_id = {study_id} and av.annotation_group_id= {annotation_group_id}
        """
    )
    annotation_df = pd.DataFrame(annotation_data)
    annotation_df.set_index("h5ad_obs_index", inplace=True)
    return annotation_df


def add_sample_annotation(adata: AnnData, annotation_df: pd.DataFrame):
    adata.obs = (
        adata.obs.reset_index()
        .join(annotation_df)
        .fillna(0)
        .astype({"annotation_value_id": "int32"})
        .astype({"annotation_value_id": "str"})
    )


diffexp_attribute = "annotation_value_id"
diff_exp_min_group_expr = 0.1
diff_exp_min_group_fc = 0.5
diff_exp_max_notgroup_expr = 1
ngenes = 100


def find_valid_attribute_values(adata: AnnData):
    valid_attribute_group_check = adata.obs[diffexp_attribute].value_counts() > 1
    attr_values = valid_attribute_group_check.index[valid_attribute_group_check].tolist()
    attr_values = [a for a in attr_values if a != "0"]
    return attr_values


def calculate_diff_exp(adata: AnnData, attr_values: List[str]):
    sc.tl.rank_genes_groups(adata, diffexp_attribute, groups=attr_values, method="wilcoxon", use_raw=False, n_genes=ngenes)
    sc.tl.filter_rank_genes_groups(
        adata,
        min_in_group_fraction=diff_exp_min_group_expr,
        min_fold_change=diff_exp_min_group_fc,
        max_out_group_fraction=diff_exp_max_notgroup_expr,
        use_raw=False,
        key="rank_genes_groups",
        key_added="rank_genes_groups_filtered",
    )
    result_dataframes = []
    for attr_value in attr_values:
        diffexp_df = sc.get.rank_genes_groups_df(adata, key="rank_genes_groups_filtered", group=attr_value)
        diffexp_df = diffexp_df[~diffexp_df["names"].isnull()]
        diffexp_df["ref_attr_value"] = attr_value
        result_dataframes.append(diffexp_df)
    diffexp_df = pd.concat(result_dataframes, axis=0).reset_index(drop=True)
    return diffexp_df


def add_omics_ids_for_names(study_id: int, diffexp_df: pd.DataFrame):
    genes_df = pd.DataFrame(
        sql_query(
            f"""select omics_id, h5ad_var_index
        from study_omics where study_id={study_id}"""
        )
    ).set_index("h5ad_var_index")
    genes_index_omics_id_df = adata.var.reset_index().join(genes_df)[["index", "omics_id"]]
    diffexp_db_df = diffexp_df.merge(genes_index_omics_id_df, left_on="names", right_on="index")
    diffexp_db_df = diffexp_db_df.dropna(subset="omics_id").astype({"omics_id": "int32", "ref_attr_value": "int32"})
    return diffexp_db_df


def save_differential_expression(study_id: int, annotation_group_id: int, diffexp_db_df: pd.DataFrame):
    for row in diffexp_db_df.to_dict(orient="records"):
        sql_query(
            f"""insert into differential_expression (study_id, omics_id, annotation_value_id, pvalue, pvalue_adj, score, log2_foldchange)
                      values ({study_id}, {row['omics_id']}, {row['ref_attr_value']}, {row['pvals']}, {row['pvals_adj']}, {row['scores']}, {row['logfoldchanges']} );""",
            fetch_results=False,
        )
    sql_query(
        f"""UPDATE study_annotation_group_ui SET differential_expression_calculated=True
                  WHERE study_id = {study_id} and annotation_group_id = {annotation_group_id};""",
        fetch_results=False,
    )

In [2]:
study_id = 3
annotation_group_id = 32
adata = read_h5ad(study_id)
adata

AnnData object with n_obs × n_vars = 44141 × 26361
    obs: 'Admission', 'ClusterID', 'DPS', 'DTF', 'Donor_full', 'HLA1', 'IFN1', 'Sex', 'Status', 'Ventilated', 'cell_type_coarse', 'cell_type_fine', 'nCount_RNA', 'nCount_SCT', 'nFeature_RNA', 'nFeature_SCT', 'percent_mt', 'percent_rpl', 'percent_rps', 'percent_rrna', 'seurat_clusters', 'singler', 'n_genes', 'CellO_celltype'
    var: 'Selected', 'sct_detection_rate', 'sct_gmean', 'sct_residual_mean', 'sct_residual_variance', 'sct_variable', 'sct_variance', 'n_cells'
    uns: 'assay', 'authors', 'cell_type_coarse_colors', 'cellenium', 'disease', 'organism', 'preprint', 'short_name', 'tissue'
    obsm: 'X_pca', 'X_umap'
    varm: 'pca_feature_loadings'

In [3]:
annotation_df = get_annotation_df(study_id, annotation_group_id)
annotation_df

,annotation_value_id
h5ad_obs_index,
0,246
3,246
6,246
17,246
18,246
...,...
12936,246
12943,246
12947,246


In [4]:
add_sample_annotation(adata, annotation_df)

In [5]:
adata.obs

,index,Admission,ClusterID,DPS,DTF,Donor_full,HLA1,IFN1,Sex,Status,...,nFeature_SCT,percent_mt,percent_rpl,percent_rps,percent_rrna,seurat_clusters,singler,n_genes,CellO_celltype,annotation_value_id
0,covid_555_1.3,ICU,19,9,9,C1 A,-0.044271,0.086385,M,COVID,...,213,2.938389,0.947867,0.663507,55.829384,18,B_cell,213,plasmablast,246
1,covid_555_1.7,ICU,10,9,9,C1 A,-0.038040,0.022590,M,COVID,...,312,10.908337,0.165906,0.041477,67.399422,9,B_cell,312,mononuclear cell,0
2,covid_555_1.8,ICU,22,9,9,C1 A,-0.043605,0.010739,M,COVID,...,336,11.203866,0.307557,0.263620,67.355011,21,B_cell,336,lymphocyte of B lineage,0
3,covid_555_1.11,ICU,30,9,9,C1 A,-0.071987,0.064483,M,COVID,...,351,5.060034,1.114923,0.686106,36.277874,29,NK_cell,351,lymphocyte of B lineage,246
4,covid_555_1.12,ICU,8,9,9,C1 A,-0.075396,0.050378,M,COVID,...,374,10.092592,0.555556,0.462963,46.481480,7,Monocyte,374,"CD14-positive, CD16-negative classical monocyte",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44136,HIP045.2543,N/A,7,0,0,H6,0.588374,0.092001,M,Healthy,...,1080,6.898247,3.637555,2.867442,10.797968,6,Monocyte,1080,"CD14-positive, CD16-negative classical monocyte",0
44137,HIP045.2544,N/A,21,0,0,H6,1.295145,0.045697,M,Healthy,...,1031,6.485527,6.157837,4.929000,12.738940,20,Monocyte,1031,mononuclear cell,0
44138,HIP045.2545,N/A,21,0,0,H6,0.892457,0.062819,M,Healthy,...,1021,4.742462,6.171483,4.978015,14.400126,20,T_cells,1021,mononuclear cell,0
44139,HIP045.2546,N/A,11,0,0,H6,0.020954,0.036953,M,Healthy,...,1063,4.573400,3.328835,2.998950,14.574899,10,Monocyte,1063,"CD14-positive, CD16-positive monocyte",0


In [6]:
attr_values = find_valid_attribute_values(adata)
attr_values

['246']

In [7]:
diffexp_df = calculate_diff_exp(adata, attr_values)
diffexp_df

,names,scores,logfoldchanges,pvals,pvals_adj,ref_attr_value
0,MTRNR2L1,26.825930,1.003452,1.610642e-158,8.491628e-155,246
1,IFI27,14.355541,1.134031,9.835531e-47,1.037098e-43,246
2,S100A8,12.439176,0.766625,1.601475e-35,1.319265e-32,246
3,IGLC3,12.022776,0.556469,2.697540e-33,1.975274e-30,246
4,CLU,11.342372,1.036954,8.092078e-30,5.078935e-27,246
5,S100A9,11.236274,0.700876,2.705619e-29,1.584952e-26,246
6,IFITM3,10.365078,0.691607,3.574653e-25,1.713299e-22,246
7,PLBD1,9.882494,0.872513,4.958318e-23,2.042285e-20,246
8,IFI44L,9.718744,0.681459,2.508557e-22,1.001941e-19,246
9,IFI6,9.661866,0.699034,4.378144e-22,1.625525e-19,246


In [8]:
diffexp_db_df = add_omics_ids_for_names(study_id, diffexp_df)
diffexp_db_df

,names,scores,logfoldchanges,pvals,pvals_adj,ref_attr_value,index,omics_id
0,MTRNR2L1,26.825930,1.003452,1.610642e-158,8.491628e-155,246,MTRNR2L1,7434
1,IFI27,14.355541,1.134031,9.835531e-47,1.037098e-43,246,IFI27,2648
2,S100A8,12.439176,0.766625,1.601475e-35,1.319265e-32,246,S100A8,174
3,IGLC3,12.022776,0.556469,2.697540e-33,1.975274e-30,246,IGLC3,2576
5,S100A9,11.236274,0.700876,2.705619e-29,1.584952e-26,246,S100A9,16226
6,IFITM3,10.365078,0.691607,3.574653e-25,1.713299e-22,246,IFITM3,2656
7,PLBD1,9.882494,0.872513,4.958318e-23,2.042285e-20,246,PLBD1,6934
9,IFI6,9.661866,0.699034,4.378144e-22,1.625525e-19,246,IFI6,2651
11,GRN,8.523890,0.546392,1.542823e-17,4.281089e-15,246,GRN,1900
13,MNDA,8.024925,0.531346,1.015880e-15,2.479595e-13,246,MNDA,15923


In [9]:
save_differential_expression(study_id, annotation_group_id, diffexp_db_df)

/var/folders/0c/8txmxvc97x5c0qhb62j88r000000gq/T/ipykernel_69284/698958756.py:16: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  r = connection.execute(text(query))
